# Interface - *neutralizing-bias*

In [1]:
import os
import re
import ast
import sys
import subprocess
import spacy
from pytorch_pretrained_bert.tokenization import BertTokenizer

NLP = spacy.load(
    "en_core_web_sm"
)  # run "python -m spacy download en_core_web_sm" to initially download the model
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased",
                                          cache_dir="cache")


05/05/2022 17:01:02 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to C:\Users\PAULPO~1\AppData\Local\Temp\tmp5kdk10nr
100%|██████████| 231508/231508 [00:00<00:00, 661157.96B/s]
05/05/2022 17:01:03 - INFO - pytorch_pretrained_bert.file_utils -   copying C:\Users\PAULPO~1\AppData\Local\Temp\tmp5kdk10nr to cache at cache\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/05/2022 17:01:03 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for cache\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/05/2022 17:01:03 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file C:\Users\PAULPO~1\AppData\Local\Temp\tmp5kdk10nr
05/05/2022 17:01:03 - INFO - pytorch_pretrained_bert.tokenization -  

In [2]:
def get_pos_dep(s: str) -> tuple[str, str]:
    """Get POS and dependency tags for a given string.
    """
    toks = s.split()

    def words_from_toks(toks):
        words = []
        word_indices = []
        for i, tok in enumerate(toks):
            if tok.startswith('##'):
                words[-1] += tok.replace('##', '')
                word_indices[-1].append(i)
            else:
                words.append(tok)
                word_indices.append([i])
        return words, word_indices

    out_pos, out_dep = [], []
    words, word_indices = words_from_toks(toks)
    analysis = NLP(' '.join(words))

    if len(analysis) != len(words):
        return None, None

    for analysis_tok, idx in zip(analysis, word_indices):
        out_pos += [analysis_tok.pos_] * len(idx)
        out_dep += [analysis_tok.dep_] * len(idx)

    assert len(out_pos) == len(out_dep) == len(toks)

    return ' '.join(out_pos), ' '.join(out_dep)

In [3]:
def tokenize(s: str):
    """BERT-tokenize a given string.
    """
    global TOKENIZER
    tok_list = TOKENIZER.tokenize(s.strip())
    return " ".join(tok_list)

In [4]:
def debias(
    input: list[str],
    concurrent: bool = False,
) -> list[dict[str, str | float]]:
    """
    Adapter for running the de-biasing script "inference.py" on the concurrent and the modular model architecture.

    Args:
        input - list of sentences to be debiased
        concurrent - run concurrent model (if False, run modular model)

    Returns:
        results - list of output dicts
    """
    assert len(input) > 0

    working_dir = "inference_concurrent" if concurrent else "inference_modular"
    inference_output = os.path.join(working_dir, "output.txt")
    test = os.path.join(working_dir, "data.txt")

    results = []

    # create input file
    with open(test, "w") as f:
        for s in input:
            tok = tokenize(s)
            pos, dep = get_pos_dep(tok)
            f.write("0\t" + (tok + "\t") * 2 + (s + "\t") * 2 + pos + "\t" +
                    dep + "\n")

    # compile command line arguments
    args = [
        sys.executable, "joint/inference.py", "--bert_full_embeddings",
        "--coverage", "--debias_weight", "1.3", "--inference_output",
        inference_output, "--pointer_generator", "--test", test,
        "--working_dir", working_dir
    ]

    if concurrent:
        args.append("--bert_encoder")
        args.append("--debias_checkpoint")
        args.append("debias_model.ckpt")
        args.append("--no_tok_enrich")

    else:
        args.append("--activation_hidden")
        args.append("--checkpoint")
        args.append("model.ckpt")
        args.append("--extra_features_top")
        args.append("--pre_enrich")
        args.append("--test_batch_size")
        args.append("1")
        args.append("--token_softmax")

    # execute shell script
    subprocess.run(" ".join(args))
    
    ## use for debugging ##
    # debug_output = subprocess.run(" ".join(args),
    #                               text=True,
    #                               capture_output=True)
    # print(debug_output.stderr)
    # print(debug_output.stdout)
    #######################

    # read output
    with open(inference_output, "r") as f:
        lines = f.readlines()
        for line in lines:
            if re.match(r'^IN SEQ:', line):
                in_seq_tok = line.split("\t")[1][3:-2]
                in_seq = in_seq_tok.replace(" ##", "")
                in_seq = re.sub(r'\s([.,;?!"])', r'\1', in_seq)

            if re.match(r'^PRED SEQ:', line):
                pred_seq_tok = line.split("\t")[1][3:-2]
                pred_seq = pred_seq_tok.replace(" ##", "")
                pred_seq = re.sub(r'\s([.,;?!"])', r'\1', pred_seq)

            if re.match(r'^PRED DIST:', line):
                pred_dist = line.split("\t")[1]
                pred_dist = ast.literal_eval(pred_dist)

                # create sequence with underlined changes
                change_seq = in_seq_tok.split()
                difs = list(
                    set(in_seq_tok.split()) - set(pred_seq_tok.split()))

                for dif in difs:
                    idx = change_seq.index(dif)

                    if pred_dist[idx] > 0.1:
                        change_seq[idx] = "\033[4m{}\033[0m".format(
                            change_seq[idx])

                change_seq = " ".join(change_seq)
                change_seq = change_seq.replace(" ##", "")
                change_seq = re.sub(r'\s([.,;?!"])', r'\1', change_seq)

                results.append({
                    "change_seq": change_seq,
                    "in_seq": in_seq,
                    "pred_seq": pred_seq,
                    "in_seq_tok": in_seq_tok,
                    "pred_seq_tok": pred_seq_tok,
                    "pred_dist": pred_dist,
                })

    return results


#### 1. Define the input sentences

In [5]:
inputs = [
    "jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.",
    "Black people are always criminal."
]

#### 2. Run the model

In [6]:
results = debias(inputs)

#### 3. Print the results

In [7]:
for result in results:
    print("Input:   {}".format(result["in_seq"]))
    print("Changes: {}".format(result["change_seq"]))
    print("Output:  {}".format(result["pred_seq"]))
    print("Bias scores: " + ("{} ({:.3f}) " * len(result["pred_dist"])).format(
        *tuple([
            tok
            for dist in zip(result["in_seq_tok"].split(), result["pred_dist"])
            for tok in dist
        ])))
    print()

Input:   jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Changes: jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other dominant peoples, religions and cultures.
Output:  jewish history is the history of the jewish people, their religion, and culture, as it developed and interacted with other peoples, religions and cultures.
Bias scores: jewish (0.015) history (0.000) is (0.014) the (0.001) history (0.002) of (0.000) the (0.001) jewish (0.130) people (0.014) , (0.000) their (0.202) religion (0.002) , (0.000) and (0.000) culture (0.000) , (0.000) as (0.000) it (0.000) developed (0.002) and (0.000) interact (0.002) ##ed (0.001) with (0.000) other (0.024) dominant (0.445) peoples (0.002) , (0.000) religions (0.000) and (0.000) cultures (0.000) . (0.000) 

Input:   black people are always criminal.
Changes: